In [28]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [29]:
length=100
batch_size=64
learning_rate=0.00001
epocas=8

In [30]:
dados_teste=np.load('dadosTeste.npy')
dadosTreinoValidacao=np.load('dadosTreinoValidacao.npy')

print(np.shape(dadosTreinoValidacao),
np.shape(dados_teste))

dadosTreinoValidacao[5200:5500]

(100000,) (8937,)


array([ 4.,  8.,  1.,  5.,  2.,  1.,  6., 10.,  7.,  4.,  3.,  0.,  4.,
        5.,  3.,  5.,  9.,  9.,  7.,  5.,  4.,  8.,  1.,  4.,  2.,  1.,
        5., 10.,  7.,  4.,  3.,  0.,  4.,  5.,  4.,  5.,  9.,  9.,  7.,
        5.,  4.,  7.,  1.,  4.,  2.,  1.,  5., 10.,  7.,  4.,  3.,  0.,
        4.,  5.,  4.,  5.,  9.,  9.,  7.,  5.,  5.,  7.,  1.,  4.,  2.,
        1.,  6., 10.,  6.,  4.,  3.,  0.,  4.,  5.,  4.,  5.,  9.,  9.,
        7.,  5.,  5.,  7.,  1.,  3.,  2.,  1.,  6., 11.,  6.,  4.,  3.,
        1.,  4.,  5.,  4.,  5.,  9.,  8.,  7.,  5.,  5.,  7.,  1.,  2.,
        2.,  2.,  7., 11.,  6.,  4.,  2.,  1.,  4.,  5.,  4.,  5.,  9.,
        8.,  7.,  5.,  4.,  7.,  1.,  3.,  3.,  2.,  7., 11.,  7.,  4.,
        2.,  2.,  4.,  5.,  4.,  5.,  9.,  7.,  7.,  5.,  4.,  7.,  1.,
        4.,  3.,  2.,  8., 12.,  7.,  4.,  2.,  2.,  4.,  5.,  4.,  5.,
        9.,  7.,  7.,  5.,  5.,  6.,  1.,  3.,  3.,  2.,  8., 12.,  7.,
        3.,  2.,  3.,  4.,  4.,  4.,  5., 10.,  7.,  8.,  6.,  6

In [31]:
percentagem_dados=80

x_train = dadosTreinoValidacao[:80000]

x_validation = dadosTreinoValidacao[80000:]

print(np.shape(x_train), np.shape(x_validation))

(80000,) (20000,)


### Dados treino

In [32]:
def preparar_targets(x_train, length,batch_size):
    d=5
    salto=3 
    n=d+salto #8
    targets=[]
    for i in range(len(x_train)):
        soma = np.sum(x_train[d:n])
        d= d+1
        n= n+1
        targets.append(soma)
    del targets[-5:]
    
    targets = np.expand_dims(targets, axis=1)
    
    new = x_train[5:]
    inputs = np.expand_dims(new, axis=1)

    time_series_data = tf.keras.preprocessing.sequence.TimeseriesGenerator(inputs, targets, length= length, batch_size=batch_size)
    
    return time_series_data


time_series_data=preparar_targets(x_train,length,batch_size)
time_series_data[0][0].shape

(64, 100, 1)

### Dados Validação

In [33]:
validation_data= preparar_targets(x_validation,length,batch_size)

validation_data[0][0].shape

(64, 100, 1)

## Rede recorrente

In [34]:
length = len(validation_data[0][0][0])
print(length)

100


In [42]:
lstm_model = keras.Sequential()

lstm_model.add(layers.LSTM(50,input_shape=(length,1),return_sequences=True, activation = 'tanh'))
lstm_model.add(layers.LSTM(50))
lstm_model.add(layers.Dense(1))

lr = learning_rate
opt = tf.keras.optimizers.Adam(lr)
lstm_model.compile(optimizer=opt, loss=tf.keras.metrics.mean_squared_error, metrics='mae') 


lstm_model.fit(time_series_data, epochs = epocas,validation_data =validation_data, shuffle= True, verbose=1)

Epoch 1/8
 705/1249 [===============>..............] - ETA: 36s - loss: 213.9247 - mae: 13.9781

KeyboardInterrupt: 

In [36]:
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 100, 50)           10400     
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [37]:
test_targets= preparar_targets(dados_teste,length,batch_size)
test_targets[0][0].shape

(64, 100, 1)

In [38]:
lstm_model.evaluate(test_targets)

138/138 [==============================] - 3s 24ms/step - loss: 22.4727 - mae: 3.8412


[22.47274398803711, 3.841202974319458]

# Tunning Trial

In [39]:
def build_model(hp):
    lstm_model = keras.Sequential()
    
    for i in range(hp.Int('layers',2,6)):
        lstm_model.add(layers.LSTM(units=hp.Int('units_'+str(i), 50,268,step=10),
                                   input_shape=(length,1),return_sequences=True, 
                                   activation=hp.Choice('activationf_'+str(i),['relu','tanh'])))
    lstm_model.add(layers.Dense(1))

    
    opt = tf.keras.optimizers.Adam(hp.Choice('lr_',[0.00001,0.000001]))
    
    
    lstm_model.compile(optimizer=opt, loss=tf.keras.metrics.mean_squared_error, metrics='mae')
    return lstm_model


In [40]:
import keras_tuner as kt
tuner3=kt.BayesianOptimization(
    build_model,objective='val_loss',max_trials=5,overwrite=True)

In [41]:
tuner3.search(time_series_data,epochs=5,validation_data=validation_data)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
3                 |?                 |layers
130               |?                 |units_0
tanh              |?                 |activationf_0
190               |?                 |units_1
relu              |?                 |activationf_1
1e-06             |?                 |lr_

Epoch 1/5
1249/1249 [==============================] - 365s 289ms/step - loss: 243.1406 - mae: 15.0260 - val_loss: 218.6479 - val_mae: 14.1315
Epoch 2/5
1119/1249 [=========================>....] - ETA: 33s - loss: 258.3994 - mae: 13.9534

KeyboardInterrupt: 

In [ ]:
tuner3.results_summary()